<a href="https://colab.research.google.com/github/Dimildizio/system_design/blob/main/create_translations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install googletrans==4.0.0-rc1   #set a PARTICULAR version omg

In [13]:
from googletrans import Translator
import pandas as pd

## Upload text

In [7]:
textwall = [
'НЕЙТРОННАЯ РЕФЛЕКТОМЕТРИЯ В РОССИИ: ТЕКУЩЕЕ СОСТОЯНИЕ И ПЕРСПЕКТИВЫ',
'В обзоре дано описание текущего состояния дел и перспектив развития в области нейтронной рефлектометрии на действующих и будущих нейтронных источниках Российской Федерации.',
'В результате ввода в эксплуатацию новых инструментов на реакторах ИР-8 и ПИК число нейтронных рефлектометров в РФ должно удвоиться.',
'В результате должен появиться набор инструментов, нацеленных на решение широкого круга задач в области физики, химии, биологии слоистых систем в интересах научного сообщества, а также для подготовки специалистов для дальнейшего развития и совершенствования данной методики.'
]

## Google translate the text

In [4]:
translator = Translator()

In [11]:
g_trans = []
for sent in textwall:
  translated = translator.translate(sent.lower(), src='ru', dest='en')
  g_trans.append(translated.text)

In [12]:
g_trans

['Neutron reflectometry in Russia: current state and prospects',
 'The review gives a description of the current state of affairs and development prospects in the field of neutron reflectometry on existing and future neutron sources of the Russian Federation.',
 'As a result of the commissioning of new tools on the IR-8 reactors and the peak, the number of neutron reflectometers in the Russian Federation should double.',
 'As a result, there should be a set of tools aimed at solving a wide range of problems in the field of physics, chemistry, biology of layered systems in the interests of the scientific community, as well as for the preparation of specialists for further development and improvement of this methodology.']